In [0]:
!nvidia-smi

!pip install pytorch-nlp
!pip install pytorch_transformers



Wed Aug 14 17:32:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Get Test Data from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!bash dropbox_uploader.sh
!echo "CHTZbyeOjyYAAAAAAAAAfZTfC_d3kTmHXSqg7-941o9Nolj-FEAJbjyUKutEtCSu" > token.txt
!bash dropbox_uploader.sh

!bash dropbox_uploader.sh download bert_imdb_test.zip
!unzip bert_imdb_test.zip
!rm bert_imdb_test.zip

!bash dropbox_uploader.sh download pretrained_bert_imdb_6000_state_dict

!bash dropbox_uploader.sh download bert_imdb_train.zip
!unzip bert_imdb_train.zip
!rm bert_imdb_train.zip

Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.19 KiB | 939.00 KiB/s, done.
Resolving deltas: 100% (505/505), done.
/content/Dropbox-Uploader

 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader8871883722667)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 This is the first time you run th

In [0]:


import numpy as np
import torch.nn as nn 

import torch
from pytorch_transformers import BertForMaskedLM, BertTokenizer, AdamW, BertModel, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from random import Random
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [0]:
#FUNCTION TO RESTRICT TRAINING DATASET
def restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels, train_labels_tensor_):
  train_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_segment_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_input_masks_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_labels_tensor = torch.empty(num_each_label*num_labels, dtype=torch.long).random_(num_labels)
  for i in range(num_labels):
  
    train_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_segment_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_segment_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_input_masks_tensor[i*num_each_label:(i+1)*num_each_label] = train_input_masks_tensor_[train_labels_tensor_==i][:num_each_label] 
 
  for i in range(num_labels):
    train_labels_tensor[i*num_each_label:(i+1)*num_each_label]=train_labels_tensor_[train_labels_tensor_==i][:num_each_label]

  return train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor

In [0]:
# import model for fine-tuning

#config = BertConfig.from_pretrained('bert-base-uncased')
#config.output_hidden_states=True
#model = torch.load('pretrained_bert_kindle_128_final', map_location='cpu')
#torch.save(model.state_dict(), 'real_model_file')
#model.load_state_dict(torch.load("pretrained_bert_imdb_512_final_state_dict", map_location='cpu'))
#model.cuda()


config = BertConfig.from_pretrained('bert-base-uncased')
config.output_hidden_states=True
model = BertForMaskedLM(config)
model.load_state_dict(torch.load("pretrained_bert_imdb_6000_state_dict", map_location='cpu'))
model.cuda()


100%|██████████| 313/313 [00:00<00:00, 117646.49B/s]


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterm

In [0]:
#Load preprocessed Data

test_ids_tensor = torch.load('bert_imdb_test/imdb_test_tensor_ids', map_location='cpu')
test_segment_ids_tensor = torch.load('bert_imdb_test/imdb_test_tensor_segments', map_location='cpu')
test_input_masks_tensor = torch.load('bert_imdb_test/imdb_test_tensor_masks', map_location='cpu')
test_labels_tensor = torch.load('bert_imdb_test/imdb_test_tensor_labels', map_location='cpu')


train_ids_tensor_ = torch.load('bert_imdb_train/imdb_train_tensor_ids', map_location='cpu')
train_segment_ids_tensor_ = torch.load('bert_imdb_train/imdb_train_tensor_segments', map_location='cpu')
train_input_masks_tensor_ = torch.load('bert_imdb_train/imdb_train_tensor_masks', map_location='cpu')
train_labels_tensor_ = torch.load('bert_imdb_train/imdb_train_tensor_labels', map_location='cpu')

In [0]:
#DATA PREP

num_train = 25000
num_test = 25000
max_length = train_ids_tensor_.shape[1]
torch.manual_seed(25082019)
num_labels = 2

#DECIDE HOW MANY OF EACH CLASS TO USE IN TRAINING DATA
restrict_training_data = 1 #Set equal to 0 or 1 
num_each_label=100

if restrict_training_data:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels,train_labels_tensor_ )
else:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, train_labels_tensor_


train_max_batch = 10
valid_max_batch = 10
test_max_batch=10

#Load data into dataloader
train_data = TensorDataset(train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_max_batch)

test_data = TensorDataset(test_ids_tensor, test_segment_ids_tensor, test_input_masks_tensor, test_labels_tensor)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=test_max_batch)




In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [0]:
#Training
#Set Loss Function
cross_entropy = nn.CrossEntropyLoss()

max_grad_norm = 1.0
# TRAIN loop
model.train()
tr_loss = 0
nb_tr_steps = 0
epochs = 5
linear_layer = nn.Linear(768,2)

for j in range(epochs):
  print("epoch: ", j+1)
  
  print("Begin Training")
  
  for step, batch in enumerate(train_dataloader):
      
      if(step%100==0):
        print("batch: ", step+1)

      # add batch to gpu
   
      batch = tuple(t.to(device) for t in batch)
      b_train_ids, b_segment_ids, b_input_masks, b_train_labels = batch
      
      
      # forward pass
      outputs = model(b_train_ids,attention_mask=b_input_masks)
      hidden_state = outputs[1][-1][:,0]
      logits = linear_layer(hidden_state)
      loss = cross_entropy(logits,b_train_labels)
      
      # backward pass
      loss.backward()

      # track train loss
      tr_loss += loss.item()
      nb_tr_steps += 1

      # gradient clipping
      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

      # update parameters
      optimizer.step()
      model.zero_grad()

      
  # print train loss per epoch
  print("Train loss: {}".format(tr_loss/nb_tr_steps))


print("Begin Evaluation")

model.eval()
total_correct = 0


for step, batch in enumerate(test_dataloader):

  if(step%100==0):
    print("batch: ", step+1)

  batch = tuple(t.to(device) for t in batch)
  b_valid_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    outputs = model(b_valid_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)

  hidden_state = outputs[1][-1][:,0]
  logits = linear_layer(hidden_state)

  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Test Acccuracy:")
print(total_correct/num_test)

            
  

epoch:  1
Begin Training
batch:  1
Train loss: 0.09793096147914185
epoch:  2
Begin Training
batch:  1
Train loss: 0.049069118315674135
epoch:  3
Begin Training
batch:  1
Train loss: 0.03275342531621087
Begin Validation
batch:  1
batch:  101
batch:  201
batch:  301
batch:  401
batch:  501
batch:  601
batch:  701
batch:  801
batch:  901
batch:  1001
batch:  1101
batch:  1201
batch:  1301
batch:  1401
batch:  1501
batch:  1601
batch:  1701
batch:  1801
batch:  1901
batch:  2001
batch:  2101
batch:  2201
batch:  2301
batch:  2401
Validation Acccuracy:
0.8934
epoch:  4
Begin Training
batch:  1
Train loss: 0.02457816293549513
epoch:  5
Begin Training
batch:  1
Train loss: 0.019672179148255965


In [0]:
model.eval()
total_correct = 0

for step, batch in enumerate(test_dataloader):

  if(step%100==0):
    print("batch: ", step+1)

  batch = tuple(t.to(device) for t in batch)
  b_valid_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    outputs = model(b_valid_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)

  hidden_state = outputs[1][-1][:,0]
  logits = linear_layer(hidden_state)

  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Test Acccuracy:")
print(total_correct/num_test)

batch:  1
batch:  101
batch:  201
batch:  301
batch:  401
batch:  501
batch:  601
batch:  701
batch:  801
batch:  901
batch:  1001
batch:  1101
batch:  1201
batch:  1301
batch:  1401
batch:  1501
batch:  1601
batch:  1701
batch:  1801
batch:  1901
batch:  2001
batch:  2101
batch:  2201
batch:  2301
batch:  2401
Validation Acccuracy:
0.89364


In [0]:
"""
model.eval()
total_correct = 0
for step, batch in enumerate(test_dataloader):

  if(step%25==0):
    print("batch: ", step+1)

  b_test_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    logits = model(b_test_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)

  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Test Acccuracy:")
print(total_correct/num_test)

"""

'\nmodel.eval()\ntotal_correct = 0\nfor step, batch in enumerate(test_dataloader):\n\n  if(step%25==0):\n    print("batch: ", step+1)\n\n  b_test_ids, b_segment_ids, b_input_masks, b_valid_labels = batch\n\n  with torch.no_grad():\n    logits = model(b_test_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)\n\n  a = logits.argmax(1)\n  b = b_valid_labels\n\n  temp_correct = torch.sum(a==b).cpu().numpy()\n  total_correct+=temp_correct\nprint("Test Acccuracy:")\nprint(total_correct/num_test)\n\n'